In [123]:
import ccxt
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [71]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'timeout': 30000,
    'enableRateLimit': True,
})
exchange.load_markets()

{'ETH/BTC': {'percentage': True,
  'tierBased': False,
  'taker': 0.001,
  'maker': 0.001,
  'precision': {'base': 8, 'quote': 8, 'amount': 3, 'price': 6},
  'limits': {'amount': {'min': 0.001, 'max': 100000.0},
   'price': {'min': 1e-06, 'max': 100000.0},
   'cost': {'min': 0.0001, 'max': None},
   'market': {'min': 0.0, 'max': 3750.34528562}},
  'id': 'ETHBTC',
  'lowercaseId': 'ethbtc',
  'symbol': 'ETH/BTC',
  'base': 'ETH',
  'quote': 'BTC',
  'baseId': 'ETH',
  'quoteId': 'BTC',
  'info': {'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': '8',
   'quoteAsset': 'BTC',
   'quotePrecision': '8',
   'quoteAssetPrecision': '8',
   'baseCommissionPrecision': '8',
   'quoteCommissionPrecision': '8',
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'isSpotTradingAllowed': True,
   'isMarginTrading

In [73]:
watching_pairs = {}
btcusdt = exchange.fetchTicker('BTC/USDT')['last']
if exchange.has['fetchOHLCV']:
    for symbol in tqdm(exchange.markets):
        if '/USDT' not in symbol:
            continue
        time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
        info = exchange.fetchTicker(symbol)
        if 50 < info['quoteVolume'] / btcusdt < 400 or info['quoteVolume'] / btcusdt > 10000:
            watching_pairs[symbol] = info['last']

100%|██████████| 1360/1360 [02:44<00:00,  8.26it/s]


In [62]:
def calculate_MA(symbol, last_price):
    ohlcv_prices = exchange.fetchOHLCV(symbol, '1d')
    ohlcv_prices = np.array(ohlcv_prices)
    MA7 = np.mean(ohlcv_prices[-7:,4])
    MA25 = np.mean(ohlcv_prices[-25:,4])
    alert_7 = False
    alert_25 = False
    if last_price > MA7*1.01 and last_price < MA7*1.01:
        alert_7 = True
    if last_price > MA25*1.01 and last_price < MA25*1.01:
        alert_25 = True
    return MA7, alert_7, MA25, alert_25

In [70]:
calculate_MA('BTC/USDT', btcusdt)

(54154.33571428571, False, 54794.8672, False)

In [124]:
ohlcv_prices = exchange.fetchOHLCV('BTC/USDT', '1d')
ohlcv_prices = np.array(ohlcv_prices)

In [112]:
close_prices = ohlcv_prices[-15:, 4]

In [120]:
diff = np.diff(close_prices)

In [114]:
rs = np.mean(diff[np.where(diff > 0)]) / np.mean(diff[np.where(diff < 0)])

In [115]:
rsi_s1 = 100 - (100/(1+rs))

In [116]:
rsi_s1

337.30474535097034

In [121]:
diff

array([-2220.08, -3363.11,  1295.55,  2012.22, -1264.81,   381.85,
          72.27,  -750.72, -3268.31,   257.64, -2037.24, -1009.87,
        3731.81,  -776.19])

In [122]:
close_prices

array([61188.39, 58968.31, 55605.2 , 56900.75, 58912.97, 57648.16,
       58030.01, 58102.28, 57351.56, 54083.25, 54340.89, 52303.65,
       51293.78, 55025.59, 54249.4 ])